In [1]:
import sys
import os
from glob import glob
sys.path.append("../NEAT")
from NEATModels import NEATDynamic, nets
from NEATModels.config import dynamic_config
from NEATUtils import helpers
from NEATUtils.helpers import load_json
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

Using TensorFlow backend.


In [2]:
imagename = '/home/sancere/Kepler/FinalONEATTraining/NEATTest/Test.tif'
model_dir = '/home/sancere/Kepler/FinalONEATTraining/Microneatmodel/'
model_name = 'microseqnetbin2d65.h5'
division_categories_json = model_dir + 'MicroscopeCategories.json'
catconfig = load_json(division_categories_json)
division_cord_json = model_dir + 'MicroscopeCord.json'
cordconfig = load_json(division_cord_json)
model = NEATDynamic(None, model_dir , model_name,catconfig, cordconfig)
n_tiles = (4,4)

event_threshold = 0.99
iou_threshold = 0.8

In [3]:
model.predict(imagename, n_tiles = n_tiles, event_threshold = event_threshold, iou_threshold = iou_threshold)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



  2%|▏         | 4/180 [00:34<24:18,  8.29s/it]

0.8560143211670177


  4%|▍         | 8/180 [01:05<22:31,  7.86s/it]

0.9104180646582198
0.9085144255632597
0.8740755815483122
0.8236734643879364
0.8549783978677176
0.8586171301743464
0.978584648424976
0.9470221276215633
0.8053446449994618
0.8503968613933836


...

0.970042818141183
0.9100018462953758
0.8991037444113459
0.82852417626617
0.8586369278005221
0.9095532762375097


  9%|▉         | 16/180 [02:06<20:52,  7.63s/it]

0.9480463597734782


 11%|█         | 19/180 [02:36<22:09,  8.25s/it]


KeyboardInterrupt: 